## Tic-Tac-Toe Agent
​
In this notebook, you will learn to build an RL agent (using Q-learning) that learns to play Numerical Tic-Tac-Toe with odd numbers. The environment is playing randomly with the agent, i.e. its strategy is to put an even number randomly in an empty cell. The following is the layout of the notebook:
        - Defining epsilon-greedy strategy
        - Tracking state-action pairs for convergence
        - Define hyperparameters for the Q-learning algorithm
        - Generating episode and applying Q-update equation
        - Checking convergence in Q-values

#### Importing libraries
Write the code to import Tic-Tac-Toe class from the environment file

In [ ]:
from TCGame_Env_Mukul import TicTacToe # - import your class from environment file
import collections
import numpy as np
import random
import pickle
import time
from matplotlib import pyplot as plt

In [ ]:
# Function to convert state array into a string to store it as keys in the dictionary
# states in Q-dictionary will be of form: x-4-5-3-8-x-x-x-x
#   x | 4 | 5
#   ----------
#   3 | 8 | x
#   ----------
#   x | x | x

def Q_state(state):

    return ('-'.join(str(e) for e in state)).replace('nan','x')

In [ ]:
# Defining a function which will return valid (all possible actions) actions corresponding to a state

def valid_actions(state):

    valid_Actions = []
    
    valid_Actions = [i for i in env.action_space(state)[0]]
    return valid_Actions

In [ ]:
# Defining a function which will add new Q-values to the Q-dictionary. 

def add_to_dict(state):
    state1 = Q_state(state)
    
    valid_act = valid_actions(state)
    if state1 not in Q_dict.keys():
        for action in valid_act:
            Q_dict[state1][action]=0

#### Epsilon-greedy strategy - Write your code here

(you can build your epsilon-decay function similar to the one given at the end of the notebook)

In [ ]:
# Defining epsilon-greedy policy.

def epsilon_greedy(state, time):
    max_epsilon = 1.0
        min_epsilon = 0.001

    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.000001*time)
    
    z = np.random.random()       
    
    if z > epsilon:
        action = max(Q_dict[Q_state(state)],key=Q_dict[Q_state(state)].get) # exploitation         
    else:
        action = random.sample(valid_actions(state),1)[0]   # exploration
    
    return action

#### Tracking the state-action pairs for checking convergence - write your code here

In [ ]:
# Initialise Q_dictionary as 'Q_dict' and States_tracked as 'States_track' (for convergence)

Q_dict = collections.defaultdict(dict)

States_track = collections.defaultdict(dict)


In [ ]:
# Initialise few random states to be tracked

def initialise_tracking_states():
    sample_qs = [('x-x-x-x-6-x-x-x-5',(2,7)),
                 ('x-x-x-x-9-x-6-x-x',(1,7)),
                 ('x-3-x-x-x-6-x-x-x',(0,1)),
                 ('x-5-x-2-x-x-4-7-x',(0,9)),
                 ('x-x-7-x-x-x-x-x-2',(1,5)),
                 ('5-x-x-x-x-6-x-x-x',(4,9))]
    
    for q_val in sample_qs:
        state = q_val[0]
        action = q_val[1]
        States_track[state][action] = []

In [ ]:
#Defining a function to save the Q-dictionary as a pickle file

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def save_tracking_states():
    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[state]:
                States_track[state][action].append(Q_dict[state][action])

In [ ]:
#Initialise tracking states

initialise_tracking_states()

#### Define hyperparameters  ---write your code here

In [ ]:
# define hyperparameters
# I have used different hyperparamters like Episode = 5m, 5.5m, 5.8m, 6m and 6.2m. LR = 0.1, 0.15, 0.2, 0.22 etc. 
# Below set are best set of HP

LR = 0.22
Gamma = 0.75
Episode = 6200000
threshold = 2500
checkpoint_print_episodes = 620000

### Q-update loop ---write your code here

In [ ]:
from tqdm import tqdm
start_time = time.time()

# Keys for random state to be tracked
track={}
track['x-x-x-x-6-x-x-x-5']=[]
track['x-x-x-x-9-x-6-x-x']=[]
track['x-5-x-2-x-x-4-7-x']=[]
track['x-3-x-x-x-6-x-x-x']=[]
track['x-x-7-x-x-x-x-x-2']=[]
track['5-x-x-x-x-6-x-x-x']=[]

# Initalizing count of game won by agent, env and tie
agent_won_cnt = 0
env_won_cnt = 0
tie_cnt = 0

for episode in tqdm(range(Episode)):
    
    env = TicTacToe()  #Calling my env class
    
    ## Initalizing parameters for the episodes
    reward=0
    curr_state = env.state
    add_to_dict(curr_state)
    is_terminal = False
    total_reward = 0
    
    while not(is_terminal):
        curr_action = epsilon_greedy(curr_state, episode) #call epsilon greedy function to get current action
    
        # if Q_state is in Q state to be track then append the action
        if Q_state(curr_state) in track.keys():
            track[Q_state(curr_state)].append(curr_action)
            
        # call step function to get next state, reward, terminal state and final flag
        next_state, reward, is_terminal, final_flag = env.step(curr_state,curr_action) 
        
        curr_state_lookup = Q_state(curr_state)
        next_state_lookup = Q_state(next_state)

        if is_terminal:
            q_value_max = 0 # Initalizing Q max
            
            # Tracking the count of games won by agent and environment
            if final_flag == "A":
                agent_won_cnt += 1
            elif final_flag == "E":
                env_won_cnt += 1
            else:
                tie_cnt += 1
        else:
            add_to_dict(next_state)
            max_next = max(Q_dict[next_state_lookup],key=Q_dict[next_state_lookup].get)
            q_value_max = Q_dict[next_state_lookup][max_next]
        
        # Q update
        Q_dict[curr_state_lookup][curr_action] += LR * ((reward + (Gamma * (q_value_max))) 
                                                        - Q_dict[curr_state_lookup][curr_action]) 
        curr_state = next_state

        total_reward += reward
    
    # print game state i.e. how many game are tie or won by env or agent
    if (episode + 1) % checkpoint_print_episodes == 0:
        print("After playing %d games, Agent Won : %.3f, Environment Won : %.3f, Tie : %.3f"% (episode + 1, 
            agent_won_cnt / (episode + 1), env_won_cnt /(episode + 1), tie_cnt / (episode + 1)))
    
    # save tracking states
    if ((episode + 1) % threshold) == 0:   
        save_tracking_states()
    
    # print when every million completed
    if ((episode + 1) % 1000000) == 0:
        print('Processed %dM episodes'%((episode+1)/1000000))
        
elapsed_time = time.time() - start_time
save_obj(States_track,'States_tracked')
save_obj(Q_dict,'Policy')

print('Total Execution time: ', elapsed_time)

#### Check the Q-dictionary

In [ ]:
#the Q-dictionary

Q_dict

In [ ]:
# length of the Q-dictionary

len(Q_dict)

In [ ]:
# try checking for one of the states - that which action your agent thinks is the best  -----This will not be evaluated

Q_dict['x-5-x-2-x-x-4-7-x']

#### Check the states tracked for Q-values convergence
(non-evaluative)

In [ ]:
# Write the code for plotting the graphs for state-action pairs tracked

plt.figure(0, figsize=(16,7))
plt.subplot(241)
t1=States_track['x-x-x-x-6-x-x-x-5'][(2,7)]
plt.title("state-action-1")
plt.plot(np.asarray(range(0, len(t1))),np.asarray(t1))

plt.subplot(242)
t2=States_track['x-x-x-x-9-x-6-x-x'][(1,7)]
plt.title("state-action-2")
plt.plot(np.asarray(range(0, len(t2))),np.asarray(t2))

plt.subplot(243)
t3=States_track['x-5-x-2-x-x-4-7-x'][(0,9)]
plt.title("state-action-3")
plt.plot(np.asarray(range(0, len(t3))),np.asarray(t3))

plt.subplot(244)
t4=States_track['x-x-7-x-x-x-x-x-2'][(1,5)]
plt.title("state-action-4")
plt.plot(np.asarray(range(0, len(t4))),np.asarray(t4))

plt.show()

### Epsilon - decay check

In [ ]:
max_epsilon = 1.0
min_epsilon = 0.001
time = np.arange(0,6200000)
epsilon = []
for i in range(0,6200000):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.000001*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()